In [44]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [45]:
df = pd.read_csv('./Data/scrapeddraft0522.csv',index_col=0)

In [46]:
df = df[df.player != '<th aria-label="Player" class="sort_default_asc show_partial_when_sorting left" data-stat="player">Player</th>']

In [47]:
df.player[df.player.str.contains('=')] = df.player[df.player.str.contains('=')].apply(lambda x: str(x).split(">")[1].split("<")[0])

In [48]:
df[['team','round','pick']] = df['draft_info'].apply(lambda x: pd.Series(str(x).split('/')[0:3]))

In [49]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x).strip()))

In [50]:
df = df[df['round'] != '']

In [51]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x)[0]))

In [52]:
df['round'][df['round']=='n'] = 8

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1221531417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'][df['round']=='n'] = 8


In [53]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x).strip().split(' ')[0]))

In [54]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x)[0:-2]))

In [55]:
df['pick'][df['pick'] == ''] = np.nan

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1104288695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pick'][df['pick'] == ''] = np.nan


In [56]:
df[['feet','inches']] = df['height'].apply(lambda x: pd.Series(str(x).split('-')))

In [57]:
df['feet'] = pd.to_numeric(df.feet,errors='coerce')
df['inches'] = pd.to_numeric(df.inches,errors='coerce')
df['height'] = (df.feet * 12) + df.inches

In [58]:
df.drop(['inches','feet'],axis=1,inplace=True)

In [59]:
df.drop(['link','draft_info'],axis = 1, inplace=True)

In [60]:
# Call NFL Players Python pacakge
player = nfl.import_players()

# Find all players who were drafted in 2005 or later
draft_players = player.loc[(player.entry_year > 2004) & (player.draft_number >= 1)].sort_values(by=['entry_year','draft_number'])

# Select relevant columns of draft players
draft_player = draft_players[['display_name','position_group','entry_year','college_conference','draft_number']]

In [61]:
cat_cols = ['player','pos','school_name','team']

In [62]:
num_cols = [col for col in df if col not in cat_cols]

In [63]:
df[num_cols] = df[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

In [64]:
drafted = df[df['round'] != 8]

In [65]:
draft = df.merge(draft_player,how='left',left_on=['year','pick'],right_on=['entry_year','draft_number'])

In [66]:
age = pd.read_excel('./Data/Age.xlsx')

In [67]:
age = age[['Year','Rnd','Pick','Age','Player']]

In [68]:
age[['Year','Rnd','Pick','Age']] = age[['Year','Rnd','Pick','Age']].apply(pd.to_numeric,errors='coerce',axis=1)

In [69]:
drafts = draft.merge(age,how='left',left_on=['year','pick','player'],right_on=['Year','Pick','Player'])

In [70]:
drafts.drop(['display_name','position_group','entry_year','draft_number','Year','Rnd','Pick','Player'],axis=1,inplace=True)

In [71]:
drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1009202155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1009202155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'


In [72]:
drafts['Round'] = drafts['round']

In [73]:
drafts.drop(['round'],axis=1,inplace=True)

In [74]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1151124999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'


In [75]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\3583540359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'


In [76]:
drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\2083932286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'


In [77]:
drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\3775298207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'


In [78]:
drafts['speed'] = (drafts.weight * 200) / (drafts.forty_yd ** 4)
drafts['shuttle_agility'] = (drafts.weight * 200) / (drafts.shuttle ** 4)
drafts['cone_agility'] = (drafts.weight * 200) / (drafts.cone ** 3)

In [79]:
drafts.Round[drafts.Round <= 3] = 2

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\874012890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round <= 3] = 2


In [80]:
drafts.Round[drafts.Round == 8] = 0

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\2272459413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round == 8] = 0


In [81]:
drafts.Round[drafts.Round >= 4] = 1

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\254994376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round >= 4] = 1


In [86]:
drafts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6067 entries, 0 to 6066
Data columns (total 91 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player                       6067 non-null   object 
 1   year                         6067 non-null   float64
 2   pos                          6067 non-null   object 
 3   school_name                  6067 non-null   object 
 4   height                       6038 non-null   float64
 5   weight                       6043 non-null   float64
 6   forty_yd                     5619 non-null   float64
 7   vertical                     4780 non-null   float64
 8   bench_reps                   4173 non-null   float64
 9   broad_jump                   4700 non-null   float64
 10  cone                         3819 non-null   float64
 11  shuttle                      3923 non-null   float64
 12  g_season                     4311 non-null   float64
 13  def_int_season    

In [82]:
drafts.to_csv('./Data/draft0522.csv')

In [83]:
drafts.to_parquet('./Data/draft0522.parquet')

In [84]:
#drafts.school_name[(drafts.college_conference.isna())].value_counts()